In [13]:
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.functions import *
from pyspark.sql.types import StructType, StructField, StringType, DateType, FloatType, IntegerType

spark = SparkSession.builder.appName("Example").getOrCreate()

#### Read Variaveis

In [14]:
ultimo_dia = "2023-12-30"

#### Base Fake

In [32]:
data_cadoc = [
    ("Sistema_1", "Empresa_1", "20230101", 1, "MODA_1", "Cd_Sub_Modd_1", "CARTAO DE CREDITO ROTATIVO' 'CARTAO DE CREDITO - ROTATIVO EM ATRASO", "Encargo_1", "Moeda_1", "Cosif_1", "1", "11111111111", "20230101", "20230110", 1000.0, 800.0, 0.05),
    ("Sistema_2", "Empresa_2", "20230102", 2, "MODA_2", "Cd_Sub_Modd_2", "CARTAO DE CREDITO ROTATIVO' 'CARTAO DE CREDITO - ROTATIVO EM ATRASO", "Encargo_2", "Moeda_2", "Cosif_2", "2", "22222222222", "20230102", "20230115", 2000.0, 1500.0, 0.06),
    ("Sistema_3", "Empresa_3", "20230103", 3, "MODA_3", "Cd_Sub_Modd_3", "CARTAO DE CREDITO ROTATIVO' 'CARTAO DE CREDITO - ROTATIVO EM ATRASO", "Encargo_3", "Moeda_3", "Cosif_3", "1", "33333333333", "20230103", "20230120", 3000.0, 2200.0, 0.07),
    ("Sistema_4", "Empresa_4", "20230104", 4, "MODA_4", "Cd_Sub_Modd_4", "CARTAO DE CREDITO ROTATIVO' 'CARTAO DE CREDITO - ROTATIVO EM ATRASO", "Encargo_4", "Moeda_4", "Cosif_4", "2", "44444444444", "20230104", "20230125", 4000.0, 3300.0, 0.08),
    ("Sistema_5", "Empresa_5", "20230105", 5, "MODA_5", "Cd_Sub_Modd_5", "CARTAO DE CREDITO ROTATIVO' 'CARTAO DE CREDITO - ROTATIVO EM ATRASO", "Encargo_5", "Moeda_5", "Cosif_5", "1", "55555555555", "20230105", "20230130", 5000.0, 4400.0, 0.09),
    ("Sistema_6", "Empresa_6", "20230106", 6, "MODA_6", "Cd_Sub_Modd_6", "CARTAO DE CREDITO ROTATIVO' 'CARTAO DE CREDITO - ROTATIVO EM ATRASO", "Encargo_6", "Moeda_6", "Cosif_6", "2", "66666666666", "20230106", "20230201", 6000.0, 5500.0, 0.1),
    ("Sistema_7", "Empresa_7", "20230107", 7, "MODA_7", "Cd_Sub_Modd_7", "CARTAO DE CREDITO ROTATIVO' 'CARTAO DE CREDITO - ROTATIVO EM ATRASO", "Encargo_7", "Moeda_7", "Cosif_7", "1", "77777777777", "20230107", "20230205", 7000.0, 6600.0, 0.11),
    ("Sistema_8", "Empresa_8", "20230108", 8, "MODA_8", "Cd_Sub_Modd_8", "CARTAO DE CREDITO ROTATIVO' 'CARTAO DE CREDITO - ROTATIVO EM ATRASO", "Encargo_8", "Moeda_8", "Cosif_8", "2", "88888888888", "20230108", "20230210", 8000.0, 7700.0, 0.12),
    ("Sistema_9", "Empresa_9", "20230109", 9, "MODA_9", "Cd_Sub_Modd_9", "CARTAO DE CREDITO ROTATIVO' 'CARTAO DE CREDITO - ROTATIVO EM ATRASO", "Encargo_9", "Moeda_9", "Cosif_9", "1", "99999999999", "20230109", "20230215", 9000.0, 8800.0, 0.13),
    ("Sistema_10", "Empresa_10", "20230110", 10, "MODA_10", "Cd_Sub_Modd_10", "CARTAO DE CREDITO ROTATIVO' 'CARTAO DE CREDITO - ROTATIVO EM ATRASO", "Encargo_10", "Moeda_10", "Cosif_10", "2", "10101010101", "20230110", "20230220", 10000.0, 9900.0, 0.14)
]

schema_cadoc = [
    "sistema", "empresa", "Dt_Ref", "idop", "moda", "submoda",
    "Mod_Bacen_3050", "Encargo_3050", "Moeda_3050", "cosif", "tp_pes", "cpf_cnpj",
    "dtop", "DTVCTO", "risco", "vlop", "TXJUR"
]

df_base = spark.createDataFrame(data_cadoc, schema_cadoc)
df_base.show(3)

+---------+---------+--------+----+------+-------------+--------------------+------------+----------+-------+------+-----------+--------+--------+------+------+-----+
|  sistema|  empresa|  Dt_Ref|idop|  moda|      submoda|      Mod_Bacen_3050|Encargo_3050|Moeda_3050|  cosif|tp_pes|   cpf_cnpj|    dtop|  DTVCTO| risco|  vlop|TXJUR|
+---------+---------+--------+----+------+-------------+--------------------+------------+----------+-------+------+-----------+--------+--------+------+------+-----+
|Sistema_1|Empresa_1|20230101|   1|MODA_1|Cd_Sub_Modd_1|CARTAO DE CREDITO...|   Encargo_1|   Moeda_1|Cosif_1|     1|11111111111|20230101|20230110|1000.0| 800.0| 0.05|
|Sistema_2|Empresa_2|20230102|   2|MODA_2|Cd_Sub_Modd_2|CARTAO DE CREDITO...|   Encargo_2|   Moeda_2|Cosif_2|     2|22222222222|20230102|20230115|2000.0|1500.0| 0.06|
|Sistema_3|Empresa_3|20230103|   3|MODA_3|Cd_Sub_Modd_3|CARTAO DE CREDITO...|   Encargo_3|   Moeda_3|Cosif_3|     1|33333333333|20230103|20230120|3000.0|2200.0| 0.07

##### Create DF Ano/Mes

In [33]:
df_cadoc_anomes = df_base.select(
    col("sistema"),
    col("empresa"),
    concat_ws(".",
              substring(col("Dt_Ref"), 7, 2),
              substring(col("Dt_Ref"), 5, 2),
              substring(col("Dt_Ref"), 1, 4)).alias("Dt_Ref"),
    col("idop").alias("Id_Oprc"),
    col("moda").alias("MODA_SAFRA"),
    col("submoda").alias("Cd_Sub_modd"),
    col("Mod_Bacen_3050"),
    col("Encargo_3050"),
    col("Moeda_3050"),
    col("cosif"),
    col("tp_pes"),
    col("cpf_cnpj"),
    concat_ws(".",
              substring(col("dtop"), 7, 2),
              substring(col("dtop"), 5, 2),
              substring(col("dtop"), 1, 4)).alias("Dt_Inic_Oprc"),
    concat_ws(".",
              substring(col("DTVCTO"), 7, 2),
              substring(col("DTVCTO"), 5, 2),
              substring(col("DTVCTO"), 1, 4)).alias("Dt_Vcmt_Oprc"),
    col("risco").alias("V_Sld_Oprc"),
    col("vlop").alias("V_Oprc"),
    col("TXJUR").alias("PC_Txa_Jurs_Eftv")
)\
.filter(col("Mod_Bacen_3050").isin("CARTAO DE CREDITO ROTATIVO' 'CARTAO DE CREDITO - ROTATIVO EM ATRASO"))

##### Create DF Ano/Mes Ant

In [34]:
df_cadoc_anomes_ant = df_base.select(
    col("sistema"),
    col("empresa"),
    concat_ws(".",
              substring(col("Dt_Ref"), 7, 2),
              substring(col("Dt_Ref"), 5, 2),
              substring(col("Dt_Ref"), 1, 4)).alias("Dt_Ref"),
    col("idop").alias("Id_Oprc"),
    col("moda").alias("MODA_SAFRA"),
    col("submoda").alias("Cd_Sub_modd"),
    col("Mod_Bacen_3050"),
    col("Encargo_3050"),
    col("Moeda_3050"),
    col("cosif"),
    col("tp_pes"),
    col("cpf_cnpj"),
    concat_ws(".",
              substring(col("dtop"), 7, 2),
              substring(col("dtop"), 5, 2),
              substring(col("dtop"), 1, 4)).alias("Dt_Inic_Oprc"),
    concat_ws(".",
              substring(col("DTVCTO"), 7, 2),
              substring(col("DTVCTO"), 5, 2),
              substring(col("DTVCTO"), 1, 4)).alias("Dt_Vcmt_Oprc"),
    col("risco").alias("V_Sld_Oprc"),
    col("vlop").alias("V_Oprc"),
    col("TXJUR").alias("PC_Txa_Jurs_Eftv")
)\
.filter(col("Mod_Bacen_3050").isin("CARTAO DE CREDITO ROTATIVO' 'CARTAO DE CREDITO - ROTATIVO EM ATRASO"))

#### Bases

In [35]:
df_cadoc_anoant1_mesant1_diaant1 = df_cadoc_anomes
df_cadoc_ano1_mes1_dia1 = df_cadoc_anomes
df_cadoc_ano2_mes2_dia2 = df_cadoc_anomes
df_cadoc_ano3_mes3_dia3 = df_cadoc_anomes
df_cadoc_ano4_mes4_dia4 = df_cadoc_anomes
df_cadoc_ano5_mes5_dia5 = df_cadoc_anomes

In [53]:
df_cadoc_anoant1_mesant1_diaant1.alias("a")\
    .join(df_cadoc_ano1_mes1_dia1.alias("b"), on="Id_Oprc", how="left")\
    .select(
            col("a.Dt_Ref"),
            col("a.Id_Oprc"),
            col("a.MODA_SAFRA"),
            col("a.Cd_Sub_Modd"),
            col("a.tp_pes"),
            col("a.cpf_cnpj"),
            col("a.Dt_Vcmt_Oprc"),
            col("a.cosif"),
            col("a.V_Oprc"),
            col("a.Pc_Txa_Jurs_Eftv"),
            date_format(to_date(col("a.Dt_Ref"), format="dd.MM.yyyy"), "dd/MM/yy").alias("data_ref"),
            date_format(to_date(col("a.Dt_Vcmt_Oprc"), format="dd.MM.yyyy"), "dd/MM/yy").alias("data_venc"),
            col("b.Dt_Ref").alias("Dt_Ref_ant"),
            col("b.Id_Oprc").alias("Id_Oprc_ant"),
            col("b.tp_pes").alias("tp_pes_ant"),
            col("b.MODA_SAFRA").alias("MODA_SAFRA_ant"),
            col("b.Cd_Sub_Modd").alias("Cd_Sub_Modd_ant"),
            col("b.Dt_Vcmt_Oprc").alias("Dt_Vcmt_Oprc_ant"),
            col("b.cosif").alias("Cosif_ant"),
            col("b.V_Oprc").alias("V_Oprc_ant"),
            col("b.Pc_Txa_Jurs_Eftv").alias("Pc_Txa_Jurs_Eftv_ant"),
            date_format(to_date(col("b.Dt_Ref"), format="dd.MM.yyyy"), "dd/MM/yy").alias("data_ref_ant"),
            date_format(to_date(col("b.Dt_Vcmt_Oprc"), format="dd.MM.yyyy"), "dd/MM/yy").alias("data_venc_ant"),
            (when(col("b.Id_Oprc")=='',0).otherwise(col("b.V_Sld_Oprc"))).cast("decimal(20,2)").alias("V_Sld_Oprc_ant"),
            col("a.V_Sld_Oprc").cast("decimal(20,2)").alias("V_Sld_Oprc"))\
     .withColumn("df1", col("V_Sld_Oprc") - col("V_Sld_Oprc_ant"))\
     .withColumn("atraso", datediff(to_date(col("data_ref"),format="dd/MM/yy"),to_date(col("data_venc"),format="dd/MM/yy")))

+----------+-------+----------+--------------+------+-----------+------------+--------+------+----------------+--------+---------+----------+-----------+----------+--------------+---------------+----------------+---------+----------+--------------------+------------+-------------+--------------+----------+----+------+
|    Dt_Ref|Id_Oprc|MODA_SAFRA|   Cd_Sub_Modd|tp_pes|   cpf_cnpj|Dt_Vcmt_Oprc|   cosif|V_Oprc|Pc_Txa_Jurs_Eftv|data_ref|data_venc|Dt_Ref_ant|Id_Oprc_ant|tp_pes_ant|MODA_SAFRA_ant|Cd_Sub_Modd_ant|Dt_Vcmt_Oprc_ant|Cosif_ant|V_Oprc_ant|Pc_Txa_Jurs_Eftv_ant|data_ref_ant|data_venc_ant|V_Sld_Oprc_ant|V_Sld_Oprc| df1|atraso|
+----------+-------+----------+--------------+------+-----------+------------+--------+------+----------------+--------+---------+----------+-----------+----------+--------------+---------------+----------------+---------+----------+--------------------+------------+-------------+--------------+----------+----+------+
|01.01.2023|      1|    MODA_1| Cd_Sub_M